In [1]:
from scipy.stats import kurtosis as kur, skew as skw, mode
#import scipy.stats as sps
from numpy import inf as npinf, nan as npnan
import pandas as pd
import gc
gc.enable()
from dask import dataframe as dd #, config as cfg
from dask.distributed import Client, LocalCluster, wait
from dask.array import stats as das
from dask_ml.decomposition import PCA
import joblib
#cfg.set({'temporary_directory': '/tmp'})


In [ ]:
def buildall(features, stats, origin, origin_name):
    if origin_name is 'src':
        opposite = 'dest'
    else:
        opposite = 'src'
        
    frames = origin.size().repartition(npartitions=1).reset_index()
    frames.columns = ['ip',origin_name+'_req']
    frames['pop_'+origin_name+'protocol'] = origin['protocol'].apply(lambda x: x.value_counts().index[0], meta=int).repartition(npartitions=1).reset_index().iloc[:,1]
    frames['nunq_'+origin_name+'protocol'] = origin['protocol'].nunique().repartition(npartitions=1).reset_index().iloc[:,1]
    frames['pop_'+origin_name+'ip'] = origin[opposite+'addr'].apply(lambda x: x.value_counts().index[0], meta=str).repartition(npartitions=1).reset_index().iloc[:,1]
    frames['nunq_'+origin_name+'ip'] = origin[opposite+'addr'].nunique().repartition(npartitions=1).reset_index().iloc[:,1]
    frames['pop_'+origin_name+'port'] = origin[origin_name+'port'].apply(lambda x: x.value_counts().index[0], meta=int).repartition(npartitions=1).reset_index().iloc[:,1]
    frames['nunq_'+origin_name+'port'] = origin[origin_name+'port'].nunique().repartition(npartitions=1).reset_index().iloc[:,1]

    
    for x,f in enumerate(features):
        if f is not 'dur_diff':
            frames[origin_name+stats[0]+f] = origin[f].max().repartition(npartitions=1).reset_index().iloc[:,1]
            frames[origin_name+stats[1]+f] = origin[f].min().repartition(npartitions=1).reset_index().iloc[:,1]
            frames[origin_name+stats[2]+f] = origin[f].mean().repartition(npartitions=1).reset_index().iloc[:,1]
            frames[origin_name+stats[3]+f] = origin[f].var().repartition(npartitions=1).reset_index().iloc[:,1]
            frames[origin_name+stats[4]+f] = origin[f].apply(skw,meta=float).repartition(npartitions=1).reset_index().iloc[:,1]
            frames[origin_name+stats[5]+f] = origin[f].apply(kur,meta=float).repartition(npartitions=1).reset_index().iloc[:,1]
        else:
            for i in ['start']:
                frames[origin_name+stats[0]+i+'_dur_diff'] = origin[i].apply(lambda x: x.diff().max(), meta=float).repartition(npartitions=1).reset_index().iloc[:,1]
                frames[origin_name+stats[1]+i+'_dur_diff'] = origin[i].apply(lambda x: x.diff().min(), meta=float).repartition(npartitions=1).reset_index().iloc[:,1]
                frames[origin_name+stats[2]+i+'_dur_diff'] = origin[i].apply(lambda x: x.diff().mean(), meta=float).repartition(npartitions=1).reset_index().iloc[:,1]
                frames[origin_name+stats[3]+i+'_dur_diff'] = origin[i].apply(lambda x: x.diff().var(), meta=float).repartition(npartitions=1).reset_index().iloc[:,1]
                frames[origin_name+stats[4]+i+'_dur_diff'] = origin[i].apply(lambda x: skw(x.diff()), meta=float).repartition(npartitions=1).reset_index().iloc[:,1]
                frames[origin_name+stats[5]+i+'_dur_diff'] = origin[i].apply(lambda x: kur(x.diff()), meta=float).repartition(npartitions=1).reset_index().iloc[:,1]
                        
    frames.name = origin_name
    return frames

def getall(src,dest,features):
    stats = ['max','min','avg','var','skew','kur']
    srcframes = buildall(features, stats, src, 'src')
    destframes = buildall(features, stats, dest, 'dest')  
    return srcframes,destframes

In [ ]:
def buildframes(features, stats, origin, origin_name):
    frames = origin.size().repartition(npartitions=1).reset_index()
    frames.columns = ['ip',origin_name+'_req']

    for x,f in enumerate(features):
        if f is not 'dur_diff':
            frames[origin_name+stats[0]+f] = origin[f].max().repartition(npartitions=1).reset_index().iloc[:,1]
            frames[origin_name+stats[1]+f] = origin[f].min().repartition(npartitions=1).reset_index().iloc[:,1]
            frames[origin_name+stats[2]+f] = origin[f].mean().repartition(npartitions=1).reset_index().iloc[:,1]
            frames[origin_name+stats[3]+f] = origin[f].var().repartition(npartitions=1).reset_index().iloc[:,1]
            frames[origin_name+stats[4]+f] = origin[f].apply(skw,meta=float).repartition(npartitions=1).reset_index().iloc[:,1]
            frames[origin_name+stats[5]+f] = origin[f].apply(kur,meta=float).repartition(npartitions=1).reset_index().iloc[:,1]
        else:
            for i in ['start']:
                frames[origin_name+stats[0]+i+'_dur_diff'] = origin[i].apply(lambda x: x.diff().max(), meta=float).repartition(npartitions=1).reset_index().iloc[:,1]
                frames[origin_name+stats[1]+i+'_dur_diff'] = origin[i].apply(lambda x: x.diff().min(), meta=float).repartition(npartitions=1).reset_index().iloc[:,1]
                frames[origin_name+stats[2]+i+'_dur_diff'] = origin[i].apply(lambda x: x.diff().mean(), meta=float).repartition(npartitions=1).reset_index().iloc[:,1]
                frames[origin_name+stats[3]+i+'_dur_diff'] = origin[i].apply(lambda x: x.diff().var(), meta=float).repartition(npartitions=1).reset_index().iloc[:,1]
                frames[origin_name+stats[4]+i+'_dur_diff'] = origin[i].apply(lambda x: skw(x.diff()), meta=float).repartition(npartitions=1).reset_index().iloc[:,1]
                frames[origin_name+stats[5]+i+'_dur_diff'] = origin[i].apply(lambda x: kur(x.diff()), meta=float).repartition(npartitions=1).reset_index().iloc[:,1]
                        
    frames.name = origin_name
    return frames
               
               
def getframes(src,dest,features):
    stats = ['max','min','avg','var','skew','kur']
    srcframes = buildframes(features, stats, src, 'src')
    destframes = buildframes(features, stats, dest, 'dest')        
    
    return srcframes,destframes

def build_reduced_frames(origin, origin_name):
    if origin_name is 'src':
        opposite = 'dest'
    else:
        opposite = 'src'
    frames = origin[opposite+'addr'].nunique().repartition(npartitions=1).reset_index()
    frames.columns = ['ip','nunq_'+origin_name+'ip']
    #frames['pop_'+origin_name+'protocol'] = origin['protocol'].apply(lambda x: x.value_counts().index[0], meta=int).repartition(npartitions=1).reset_index().iloc[:,1]
    #frames['pop_'+origin_name+'ip'] = origin[opposite+'addr'].apply(lambda x: x.value_counts().index[0], meta=str).repartition(npartitions=1).reset_index().iloc[:,1]
    #frames['nunq_'+origin_name+'port'] = origin[origin_name+'port'].nunique().repartition(npartitions=1).reset_index().iloc[:,1]
    #frames['pop_'+origin_name+'port'] = origin[origin_name+'port'].apply(lambda x: x.value_counts().index[0], meta=int).repartition(npartitions=1).reset_index().iloc[:,1]

    return frames

def get_reduced_frames(src,dest):
    srcframes = build_reduced_frames(src, 'src')
    destframes = build_reduced_frames(dest, 'dest')        
    
    return srcframes,destframes
    

In [ ]:
if 1:
    inputfile = '/data/maxim/data/raw_flow_w_rate.csv'
    df = dd.read_csv(inputfile)
else:
    inputfile = '/data/maxim/data/raw_flow_test.csv'
    df = dd.read_csv(inputfile, blocksize='2MB')

print(df.shape)
print(df.npartitions)

In [ ]:
try:
    with LocalCluster(processes=True,n_workers=5,threads_per_worker=2) as cluster:
        cluster.adapt(minimum_cores=10,maximum_cores=10,maximum_memory="32GB")
        with Client(cluster) as client:
            new =  dd.concat([df["srcaddr"].drop_duplicates(),df['destaddr'].drop_duplicates()], axis=0).drop_duplicates().to_frame()
            new.columns = ["ip"]
            newest = new.set_index('ip')
            newest = client.persist(newest)
            newest.name = 'newest'
            wait(newest)
            src = df.groupby(['srcaddr'],sort=True)
            dest = df.groupby(['destaddr'],sort=True)
            features = ['bytes','packets','duration','rate', 'dur_diff']
            srcframes, destframes = getall(src,dest,features)
            frames = dd.merge(srcframes,destframes, how='outer',on='ip')
            newest = dd.merge(newest,frames,how='left',left_index=True, right_on='ip').reset_index(drop=True)
            newest['ratio'] = (newest['src_req']/newest['dest_req']).round(decimals=3)
            newest = newest.replace([npinf, -npinf], npnan).fillna(0)
            newest = client.persist(newest)
            wait(newest)
            df = client.compute(newest).result()
            
    print('finished try')
    df.to_csv('raw_results_t.csv', index=False)

except:
    print('started except')
    with LocalCluster(processes=True,n_workers=3,threads_per_worker=3) as cluster:
        cluster.adapt(minimum_cores=9,maximum_cores=9,maximum_memory="32GB")
        with Client(cluster) as client:
            new =  dd.concat([df["srcaddr"].drop_duplicates(),df['destaddr'].drop_duplicates()], axis=0).drop_duplicates().to_frame()
            new.columns = ["ip"]
            newest = new.set_index('ip')
            newest = client.persist(newest)
            newest.name = 'newest'
            wait(newest)
            src = df.groupby(['srcaddr'],sort=True)
            dest = df.groupby(['destaddr'],sort=True)
            features = ['bytes','packets','duration','rate', 'dur_diff']
            srcframes, destframes = getframes(src,dest,features)
            frames = dd.merge(srcframes,destframes, how='outer',on='ip')
            newest = dd.merge(newest,frames,how='left',left_index=True, right_on='ip').reset_index(drop=True)
            newest['ratio'] = (newest['src_req']/newest['dest_req']).round(decimals=3)
            newest = newest.replace([npinf, -npinf], npnan).fillna(0)
            newest = client.persist(newest)
            wait(newest)
            df_origin = client.compute(newest).result()
            wait(df_origin)
    df_origin.to_csv('origin_results.csv', index=False)
    with LocalCluster(processes=True,n_workers=5,threads_per_worker=2) as cluster:
        cluster.adapt(minimum_cores=10,maximum_cores=10,maximum_memory="32GB")
        with Client(cluster) as client:
            new =  dd.concat([df["srcaddr"].drop_duplicates(),df['destaddr'].drop_duplicates()], axis=0).drop_duplicates().to_frame()
            new.columns = ["ip"]
            newest = new.set_index('ip')
            newest = client.persist(newest)
            newest.name = 'newest'
            wait(newest)
            src = df.groupby(['srcaddr'],sort=True)
            dest = df.groupby(['destaddr'],sort=True)
            srcframes, destframes = get_reduced_frames(src,dest)
            frames = dd.merge(srcframes,destframes, how='outer',on='ip')
            frames = frames.replace([npinf, -npinf], npnan).fillna(0)    
            newest = client.persist(newest)
            wait(newest)
            df_new = client.compute(newest).result()
    df = pd.merge(df_origin, df_new, on='ip')
    df.to_csv('raw_results_e.csv', index=False)
    print('finished except')
    

In [ ]:
with LocalCluster(processes=True,n_workers=5,threads_per_worker=2) as cluster:
        cluster.adapt(minimum_cores=9,maximum_cores=9,maximum_memory="32GB")
        with Client(cluster) as client:
            new =  dd.concat([df["srcaddr"].drop_duplicates(),df['destaddr'].drop_duplicates()], axis=0).drop_duplicates().to_frame()
            new.columns = ["ip"]
            newest = new.set_index('ip')
            newest = client.persist(newest)
            newest.name = 'newest'
            wait(newest)
            src = df.groupby(['srcaddr'],sort=True)
            dest = df.groupby(['destaddr'],sort=True)
            srcframes, destframes = get_reduced_frames(src,dest)
            frames = dd.merge(srcframes,destframes, how='outer',on='ip')
            newest = dd.merge(newest,frames,how='left',left_index=True, right_on='ip').reset_index(drop=True)
            newest = newest.replace([npinf, -npinf], npnan).fillna(0)    
            newest = client.persist(newest)
            wait(newest)
            df_new = client.compute(newest).result()
            df_new.to_csv('new_results.csv', index=False)

In [ ]:
df_new.to_csv('new_results.csv', index=False)

In [ ]:
df_origin = pd.read_csv('origin_results.csv')
df_new
df = pd.merge(df_origin, df_new, on='ip')


In [ ]:
df.columns


In [ ]:
results = pd.read_csv("hosts.csv")
results = results.drop(list(((df.columns).intersection(results.columns))[1:]), axis=1)
df = pd.merge(df,results, on='ip')


In [ ]:
df.to_csv('final.csv')

In [ ]:
df

In [ ]:
import numpy as np
from multiprocessing import cpu_count, Pool
from netaddr import IPNetwork, IPAddress
from ipwhois import IPWhois
import swifter
import tqdm
import socket,struct
import pandas as pd
import gc
gc.enable()

def check_in_subnets(ip, subnets):
    ip = IPAddress(ip)
    for subnet in subnets:
        if ip in subnet:
            return True
    return False

def private_subnets():
    ip = ['10.0.0.0','172.16.0.0','192.168.0.0']
    mask = [8,12,16]
    for i in range(len(ip)):
        ip[i] = IPNetwork(ip[i] + '/' + str(mask[i]))
    return ip

def cu_subnets():
    subnets = []
    with open('hosts.txt') as reader:
        for line in reader.readlines():
            l = line.rstrip().split(" | ")
            l[0] = IPNetwork(l[0])
            subnets.append(l)
    subnets.reverse()
    return subnets

def check_in_subnets_ret_name(ipstr, subnets):
    ip = IPAddress(str(ipstr))
    for subnet in subnets:
        if ip in subnet[0]:
            return subnet[1]
    return False

def whois(x):
    x = str(x).rstrip()
    s = check_in_subnets_ret_name(x,subs)
    if(s != False):
        return s
    elif(check_in_subnets(x, p)):
        return "private"
    elif(x == '0'):
        return x
    else:
        try:
            obj = IPWhois(x)
            results = obj.lookup_whois()
            l = [IPNetwork(results['asn_cidr']), results['nets'][0]['name'] + " - " + results['nets'][0]['description']]
            subs.append(l)
            return results['nets'][0]['name'] + " - " + results['nets'][0]['description']
        except:
            try:
                return socket.gethostbyaddr(x)
            except:
                return "n/a"
        

def whoiscu(x):
    x = str(x).rstrip()
    s = check_in_subnets_ret_name(x,subs)
    if(s != False):
        return s
    elif(check_in_subnets(x, p)):
        return "private"
    elif(x == '0'):
        return x
    else:
        return 'external'
            
def iscu(x):
    x = str(x).rstrip()
    s = check_in_subnets(x, subs)
    if(s != False):
        return True
    elif(check_in_subnets(x, p)):
        return True
    else:
        return False

def parallelize(data, func):
    pool = Pool(processes = 10)
    res = [None]*len(data)
    i=0
    for x in tqdm.tqdm(pool.imap_unordered(func, data), total=len(data)):
        res[i] = x
        i += 1
    pool.close()
    pool.join()
    return res

In [ ]:
p = private_subnets()
subs = cu_subnets()
df = pd.read_csv('final.csv')
df['internal'] = df['ip'].swifter.allow_dask_on_strings().apply(iscu)

In [ ]:
mask = df.internal == True
df['host'] = 'external'
df['pop_srchost'] = 'external'
df['pop_desthost'] = 'external'
df.loc[mask,'host'] = df.loc[mask,'ip'].swifter.allow_dask_on_strings().apply(whoiscu)
df.loc[mask,'pop_srchost'] = df.loc[mask,'pop_srcip'].swifter.allow_dask_on_strings().apply(whoiscu)
df.loc[mask,'pop_desthost'] = df.loc[mask,'pop_destip'].swifter.allow_dask_on_strings().apply(whoiscu)
#df['pop_srchost'] = df['pop_srcip'].swifter.allow_dask_on_strings().apply(whoiscu)
#df['pop_desthost'] = df['pop_destip'].swifter.allow_dask_on_strings().apply(whoiscu)

In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df

In [ ]:
col = df.pop('host')
df.insert(0, 'host', col)
col = df.pop('pop_srchost')
df.insert(2, 'pop_srchost', col)
col = df.pop('pop_desthost')
df.insert(3, 'pop_desthost', col)
col = df.pop('dest_req')
df.insert(5, 'dest_req', col)
col = df.pop('pop_srcport')
df.insert(6, 'pop_srcport', col)
col = df.pop('pop_destport')
df.insert(7, 'pop_destport', col)
col = df.pop('pop_srcprotocol')
df.insert(8, 'pop_srcprotocol', col)
col = df.pop('pop_destprotocol')
df.insert(9, 'pop_destprotocol', col)
pd.options.display.max_columns = 80
df

In [ ]:
df.to_csv('final_hosts.csv', index=False)

In [ ]:
print('h')